In [1]:
import re
import requests
from html.parser import HTMLParser

from time import sleep
from tqdm import tqdm

In [6]:
#手工输入会议以及链接
# """GECCO, SSBSE, QRS还没找到"""

journals = []
confs = []
for line in open("conf_list.csv"):
    line = line.replace("http://","https://").replace("/index.html","/")
    data = line.split("\t")
    if data[1] == "journal":
        journals.append((data[0],data[2].replace("\n","")))
    elif data[1] == "conf":
        confs.append((data[0],data[2].replace("\n","")))
    print(line.split("\t"))

['TOSEM', 'journal', 'https://dblp.uni-trier.de/db/journals/tosem/\n']
['TSE', 'journal', 'https://dblp.uni-trier.de/db/journals/tse/\n']
['IST', 'journal', 'https://dblp.uni-trier.de/db/journals/infsof/\n']
['JSS', 'journal', 'https://dblp.uni-trier.de/db/journals/jss/\n']
['ESE', 'journal', 'https://dblp.uni-trier.de/db/journals/ese/\n']
['SPE', 'journal', 'https://dblp.uni-trier.de/db/journals/spe/\n']
['STVR', 'journal', 'https://dblp.uni-trier.de/db/journals/stvr/\n']
['ASE journals', 'journal', 'https://dblp.uni-trier.de/db/journals/ase/\n']
['ICSE', 'conf', 'https://dblp.uni-trier.de/db/conf/icse/\n']
['FSE', 'conf', 'https://dblp.uni-trier.de/db/conf/sigsoft/\n']
['ASE conf', 'conf', 'https://dblp.uni-trier.de/db/conf/kbse/\n']
['ISSTA', 'conf', 'https://dblp.uni-trier.de/db/conf/issta/\n']
['ECOOP', 'conf', 'https://dblp.uni-trier.de/db/conf/ecoop/\n']
['ICPC', 'conf', 'https://dblp.uni-trier.de/db/conf/iwpc/\n']
['SANER', 'conf', 'https://dblp.uni-trier.de/db/conf/wcre/\n']
[

In [ ]:
#先按照链接获取到dblp里面对应的journal所有volume的链接
#再进入各个卷的链接，获取全部文章

In [7]:
from bs4 import BeautifulSoup 
import re

volums_jour = []
for journal, link in tqdm(journals,"Journals"):
    r = requests.get(link, headers={"Connection": "close"})
    if r.status_code == 200:
        html = r.text
    else:
        print(r.status_code)
    r.close()
    sleep(0.01)
    
    bs = BeautifulSoup(html,"html.parser")
    for item in bs.find_all("li"):
        if item.attrs == {} and item.a != None and item.img == None and item.a.attrs['href'].startswith(link):
            year = re.findall('\d{4}', item.text.split('\n')[0], flags=0)[0]
            for a in item.find_all('a'):
                vol = re.findall('\d+', a.text.replace(year, ""), flags=0)[0]
                volums_jour.append((journal, year, vol, a.attrs['href']))

Journals: 100%|██████████████████████████████████████████████████████████████████████████| 8/8 [00:17<00:00,  2.21s/it]


In [8]:
jour_paper_list = []
for conf, year, conf_det, href in tqdm(volums_jour,"Journal"):
    r = requests.get(href, headers={"Connection": "close"})
    if r.status_code == 200:
        html = r.text
    else:
        print(r.status_code)
    r.close()
    sleep(0.01)

    bs = BeautifulSoup(html,"html.parser")
    first = True
    for item in bs.find_all("cite"):
        for title in item.find_all("span", "title"):
            if first:
                first = False
            else:
                jour_paper_list.append((conf, year, conf_det, title.text))

Journal: 100%|███████████████████████████████████████████████████████████████████████| 495/495 [12:21<00:00,  1.50s/it]


In [ ]:
#先按照链接获取到dblp里面对应的conference所有track的链接
#再进入各个卷的链接，获取全部文章

In [9]:
from bs4 import BeautifulSoup 
import re

volums_conf = []
for conf, link in tqdm(confs,"Conferences"):
    r = requests.get(link, headers={"Connection": "close"})
    if r.status_code == 200:
        html = r.text
    else:
        print(r.status_code)
    r.close()
    sleep(0.01)
    
    bs = BeautifulSoup(html,"html.parser")
    for item in bs.find_all("cite"):
        if item.find("span", itemprop="publisher"):
            publisher = item.find("span", itemprop="publisher").text
        else:
#             print(item)
            break
        year = item.find("span", itemprop="datePublished").text

        for title in item.find_all("span", "title"):
            href = list(title.next_siblings)[-1].attrs["href"]
        volums_conf.append((conf, year, publisher, title.text, href))

Conferences: 100%|█████████████████████████████████████████████████████████████████████| 20/20 [00:49<00:00,  2.46s/it]


In [10]:
conf_paper_list = []
for conf, year, publisher, conf_det, href in tqdm(volums_conf,"Conferences"):
    r = requests.get(href, headers={"Connection": "close"})
    if r.status_code == 200:
        html = r.text
    else:
        print(r.status_code)
    r.close()
    sleep(0.01)

    bs = BeautifulSoup(html,"html.parser")
    first = True
    for item in bs.find_all("cite"):
        for title in item.find_all("span", "title"):
            if first:
                first = False
            else:
                conf_paper_list.append((conf, year, publisher, conf_det, title.text))

Conferences: 100%|███████████████████████████████████████████████████████████████████| 532/532 [15:43<00:00,  1.77s/it]


In [11]:
len(jour_paper_list)

17653

In [12]:
len(conf_paper_list)

33787

In [13]:
#储存全部的文章列表

In [14]:
writer = open("conf_paper_list.csv", 'w', encoding="utf-8")
for conf, year, publisher, conf_det, title in conf_paper_list:
    line = "{}\t{}\t{}\t{}\t{}\n".format(conf, year, publisher, conf_det, title)
    writer.write(line)
writer.close()

In [15]:
writer = open("jour_paper_list.csv", 'w', encoding="utf-8")
for conf, year, conf_det, title in jour_paper_list:
    line = "{},{},{},{}\n".format(conf, year, conf_det, title)
    writer.write(line)
writer.close()

In [16]:
conf_paper_list = []
for line in open("conf_paper_list.csv", encoding="utf-8"):
    line = line.replace("\n","")
    
    conf = line.split("\t")[0]
    year = line.split("\t")[1]
    publisher = line.split("\t")[2]
    conf_det = line.split("\t")[3]
    title = line.split("\t")[-1]
    conf_paper_list.append((conf, year, publisher, conf_det, title))

In [17]:
jour_paper_list = []
for line in open("jour_paper_list.csv", encoding="utf-8"):
    line = line.replace("\n","")
    
    conf = line.split(",")[0]
    year = line.split(",")[1]
    conf_det = line.split(",")[2]
    title = line.split(",")[3]
    jour_paper_list.append((conf, year, conf_det, title))

In [18]:
#筛选并存储

In [19]:
keywords = [
    "deep learning",  
    "neural network",
    "dnn", 
    "cnn",
    "machine learning",
    
    "autonomous vehicle", 
    "autonomous driving",
    "self-driving", 
    "automatics",
    "autonomous car",
    "driver assistance",
    "adas",
    "self driving",
    "overtaking",
    "automated vehicle",
    "automated driving",
    "automated car",
    "driving scenarios",
    "vehicle safety",
    "intelligent vehicle",
    "intelligent car",
    "apollo",
    "waymo",
    "vehicle signalization system",
    "pedestrian",
    "lane",
    "carla",
    "lgsvl",
    "vehicle",
    "v2x",
    "autonomous electric vehicle",
    "autonomous electric car",
    "connected vehicle",
    "object detection",
    "lidar",
    "traffic scene",
    "traffic accident",
    "driving simulator",
    "vision-based control systems"
]
keywords_dict = {}
for keyword in keywords:
    keywords_dict[keyword] = 0

In [20]:
# keywords = [
#     "autonomous vehicle", 
#     "autonomous driving",
#     "self-driving", 
#     "automatics",
#     "autonomous car",
#     "driver assistance",
#     "adas",
#     "self driving",
#     "vechicle",

#     "overtaking",
#     "automated vehicle",
#     "automated driving",
#     "automated car",
#     "driving scenarios",
#     "vehicle safety",
#     "intelligent vehicle",
#     "intelligent car",
#     "apollo",
#     "waymo",
#     "vehicle signalization system",
#     "pedestrian",
#     "carla",
#     "lgsvl",
#     "v2x",
#     "autonomous electric vehicle",
#     "autonomous electric car",
#     "connected vehicle",
#     "object detection",
#     "traffic scene",
#     "traffic accident",
#     "driving simulator",
#     "vision-based control systems"
# ]
# keywords_dict = {}
# for keyword in keywords:
#     keywords_dict[keyword] = 0

In [21]:
conf_paper_filt = []
for conf, year, publisher, conf_det, title in conf_paper_list:
    if int(year) > 2015:
        filted = True
        for keyword in keywords:
            if keyword in title.lower():
                filted = False
                for words in ["by deep learning", "using deep learning", 
                             "by machine learning", "using machine learning", 
                             "by cnn", "using cnn", 
                             "by dnn", "using dnn"]:
                    if words in title.lower():
                        filted = True
                    if not filted:
                        keywords_dict[keyword] += 1
        if not filted:
            conf_paper_filt.append((conf, year, publisher, conf_det, title))

In [22]:
writer = open("conf_paper_filt.tsv", 'w', encoding="utf-8")
for conf, year, publisher, conf_det, title in conf_paper_filt:
    line="{}\t{}\t{}\t{}\t{}\n".format(conf, year, publisher, conf_det, title)
    writer.write(line)
writer.close()

In [23]:
len(conf_paper_filt)

536

In [24]:
jour_paper_filt = []
for jour, year, jour_det, title in jour_paper_list:
    if int(year) > 2015:
        filted = True
        for keyword in keywords:
            if keyword in title.lower():
                filted = False
                for words in ["by deep learning", "using deep learning", 
                             "by machine learning", "using machine learning", 
                             "by cnn", "using cnn", 
                             "by dnn", "using dnn"]:
                    if words in title.lower():
                        filted = True
                    if not filted:
                        keywords_dict[keyword] += 1
        if not filted:
            jour_paper_filt.append((jour, year, jour_det, title))

In [25]:
# jour_paper_filt = []
# for jour, year, jour_det, title in jour_paper_list:
#     if int(year) > 2015:
#         filted = True
#         for keyword in keywords:
#             if keyword in title.lower():
#                 filted = False
#                 keywords_dict[keyword] += 1
#         if not filted:
#             jour_paper_filt.append((jour, year, jour_det, title))

In [26]:
writer = open("jour_paper_filt.tsv", 'w', encoding="utf-8")
for jour, year, conf_det, title in jour_paper_filt:
    line="{}\t{}\t{}\t{}t\n".format(jour, year, jour_det, title)
    writer.write(line)
writer.close()

In [27]:
len(jour_paper_filt)

77

In [28]:
keywords_dict

{'deep learning': 1032,
 'neural network': 1616,
 'dnn': 128,
 'cnn': 120,
 'machine learning': 1282,
 'autonomous vehicle': 128,
 'autonomous driving': 160,
 'self-driving': 40,
 'automatics': 0,
 'autonomous car': 24,
 'driver assistance': 8,
 'adas': 8,
 'self driving': 0,
 'overtaking': 0,
 'automated vehicle': 24,
 'automated driving': 48,
 'automated car': 0,
 'driving scenarios': 8,
 'vehicle safety': 0,
 'intelligent vehicle': 0,
 'intelligent car': 0,
 'apollo': 8,
 'waymo': 0,
 'vehicle signalization system': 0,
 'pedestrian': 41,
 'lane': 80,
 'carla': 0,
 'lgsvl': 0,
 'vehicle': 464,
 'v2x': 8,
 'autonomous electric vehicle': 0,
 'autonomous electric car': 0,
 'connected vehicle': 8,
 'object detection': 24,
 'lidar': 0,
 'traffic scene': 0,
 'traffic accident': 0,
 'driving simulator': 0,
 'vision-based control systems': 8}

In [29]:
writer = open("keyword.tsv","w",encoding="utf-8")
for keyword in keywords_dict:
    writer.write("{}\t{}\n".format(keyword, keywords_dict[keyword]))
writer.close()